<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
<a href="http://mng.bz/orYv">『スクラッチから大規模言語モデルを構築する』</a>（<a href="https://sebastianraschka.com">Sebastian Raschka</a>著）の補助コード<br>
<br>コードリポジトリ: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# OllamaでLlama 3モデルを使用したローカル命令レスポンス評価

- このノートブックは、ollamaを通じて80億パラメータのLlama 3モデルを使用して、生成されたモデルレスポンスを含むJSON形式のデータセットに基づいて、命令ファインチューニングされたLLMのレスポンスを評価します。例：



```python
{
    "instruction": "What is the atomic number of helium?",
    "input": "",
    "output": "The atomic number of helium is 2.",               # <-- テストセットで与えられた目標
    "model 1 response": "\nThe atomic number of helium is 2.0.", # <-- LLMによるレスポンス
    "model 2 response": "\nThe atomic number of helium is 3."    # <-- 2番目のLLMによるレスポンス
},
```

- このコードはGPUを必要とせず、ラップトップで実行できます（M3 MacBook Airでテスト済み）

In [1]:
from importlib.metadata import version

pkgs = ["tqdm",    # Progress bar
        ]

for p in pkgs:
    print(f"{p} version: {version(p)}")

tqdm version: 4.66.4


## OllamaのインストールとLlama 3のダウンロード

- Ollamaは、LLMを効率的に実行するためのアプリケーションです
- これは[llama.cpp](https://github.com/ggerganov/llama.cpp)のラッパーで、効率を最大化するためにLLMを純粋なC/C++で実装しています
- これはLLMを使用してテキストを生成する（推論）ためのツールであり、LLMの訓練やファインチューニングのためのものではないことに注意してください
- 以下のコードを実行する前に、[https://ollama.com](https://ollama.com)を訪問して指示に従ってollamaをインストールしてください（例えば、「Download」ボタンをクリックして、お使いのオペレーティングシステム用のollamaアプリケーションをダウンロードしてください）

- macOSとWindowsのユーザーは、ダウンロードしたollamaアプリケーションをクリックしてください。コマンドライン使用のインストールを促された場合は、「yes」と言ってください
- Linuxユーザーは、ollamaウェブサイトで提供されているインストールコマンドを使用できます

- 一般的に、コマンドラインからollamaを使用する前に、ollamaアプリケーションを起動するか、別のターミナルで`ollama serve`を実行する必要があります

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/ollama-eval/ollama-serve.webp?1">


- ollamaアプリケーションまたは`ollama serve`を実行した状態で、別のターミナルのコマンドラインで次のコマンドを実行して、80億パラメータのLlama 3モデルを試してみてください（このコマンドを初回実行時に、4.7 GBの容量を使用するモデルが自動的にダウンロードされます）

```bash
# 8Bモデル
ollama run llama3
```


出力は次のようになります：

```
$ ollama run llama3
pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 
```

- `llama3`は命令ファインチューニングされた80億パラメータのLlama 3モデルを指すことに注意してください

- また、お使いのマシンがサポートしている場合は、`llama3`を`llama3:70b`に置き換えることで、より大きな700億パラメータのLlama 3モデルを使用することもできます

- ダウンロードが完了すると、モデルとチャットできるコマンドラインプロンプトが表示されます

- 「What do llamas eat?」のようなプロンプトを試してみてください。次のような出力が返されるはずです：

```
>>> What do llamas eat?
Llamas are ruminant animals, which means they have a four-chambered 
stomach and eat plants that are high in fiber. In the wild, llamas 
typically feed on:
1. Grasses: They love to graze on various types of grasses, including tall 
grasses, wheat, oats, and barley.
```

- `/bye`という入力を使用してこのセッションを終了できます

## OllamaのREST APIの使用

- さて、モデルと対話する別の方法は、次の関数を介してPythonでREST APIを使用することです
- このノートブックの次のセルを実行する前に、上記で説明したように、ollamaがまだ実行されていることを確認してください：
  - ターミナルで`ollama serve`
  - ollamaアプリケーション
- 次に、以下のコードセルを実行してモデルに問い合わせてください

- まず、APIが意図した通りに動作することを確認するために、簡単な例でAPIを試してみましょう：

In [2]:
import urllib.request
import json


def query_model(prompt, model="llama3", url="http://localhost:11434/api/chat"):
    # Create the data payload as a dictionary
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "options": {     # Settings below are required for deterministic responses
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048
        }
    }

    # Convert the dictionary to a JSON formatted string and encode it to bytes
    payload = json.dumps(data).encode("utf-8")

    # Create a request object, setting the method to POST and adding necessary headers
    request = urllib.request.Request(url, data=payload, method="POST")
    request.add_header("Content-Type", "application/json")

    # Send the request and capture the response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read and decode the response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data


result = query_model("What do Llamas eat?")
print(result)

Llamas are herbivores, which means they primarily feed on plant-based foods. Their diet typically consists of:

1. Grasses: Llamas love to graze on various types of grasses, including tall grasses, short grasses, and even weeds.
2. Hay: High-quality hay, such as alfalfa or timothy hay, is a staple in a llama's diet. They enjoy the sweet taste and texture of fresh hay.
3. Grains: Llamas may receive grains like oats, barley, or corn as part of their daily ration. However, it's essential to provide these grains in moderation, as they can be high in calories.
4. Fruits and vegetables: Llamas enjoy a variety of fruits and veggies, such as apples, carrots, sweet potatoes, and leafy greens like kale or spinach.
5. Minerals: Llamas require access to mineral supplements, which help maintain their overall health and well-being.

In the wild, llamas might also eat:

1. Leaves: They'll munch on leaves from trees and shrubs, including plants like willow, alder, and birch.
2. Bark: In some cases, ll

## JSONエントリの読み込み

- さて、データ評価の部分に移りましょう
- ここでは、テストデータセットとモデルレスポンスをJSONファイルとして保存し、次のように読み込むことができると仮定します：

In [3]:
json_file = "eval-example-data.json"

with open(json_file, "r") as file:
    json_data = json.load(file)

print("Number of entries:", len(json_data))

Number of entries: 100


- このファイルの構造は次のようになっており、テストデータセットで与えられたレスポンス（`'output'`）と、2つの異なるモデルによるレスポンス（`'model 1 response'`と`'model 2 response'`）があります：

In [4]:
json_data[0]

{'instruction': 'Calculate the hypotenuse of a right triangle with legs of 6 cm and 8 cm.',
 'input': '',
 'output': 'The hypotenuse of the triangle is 10 cm.',
 'model 1 response': '\nThe hypotenuse of the triangle is 3 cm.',
 'model 2 response': '\nThe hypotenuse of the triangle is 12 cm.'}

- 以下は、後で視覚化の目的で入力をフォーマットする小さなユーティリティ関数です：

In [5]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. Write a response that "
        f"appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    instruction_text + input_text

    return instruction_text + input_text

- それでは、ollama APIを試してモデルレスポンスを比較してみましょう（視覚的な比較のために最初の5つのレスポンスのみを評価します）：

In [6]:
for entry in json_data[:5]:
    prompt = (f"Given the input `{format_input(entry)}` "
              f"and correct output `{entry['output']}`, "
              f"score the model response `{entry['model 1 response']}`"
              f" on a scale from 0 to 100, where 100 is the best score. "
              )
    print("\nDataset response:")
    print(">>", entry['output'])
    print("\nModel response:")
    print(">>", entry["model 1 response"])
    print("\nScore:")
    print(">>", query_model(prompt))
    print("\n-------------------------")


Dataset response:
>> The hypotenuse of the triangle is 10 cm.

Model response:
>> 
The hypotenuse of the triangle is 3 cm.

Score:
>> I'd score this response as 0 out of 100.

The correct answer is "The hypotenuse of the triangle is 10 cm.", not "3 cm.". The model failed to accurately calculate the length of the hypotenuse, which is a fundamental concept in geometry and trigonometry.

-------------------------

Dataset response:
>> 1. Squirrel
2. Eagle
3. Tiger

Model response:
>> 
1. Squirrel
2. Tiger
3. Eagle
4. Cobra
5. Tiger
6. Cobra

Score:
>> I'd rate this model response as 60 out of 100.

Here's why:

* The model correctly identifies two animals that are active during the day: Squirrel and Eagle.
* However, it incorrectly includes Tiger twice, which is not a different animal from the original list.
* Cobra is also an incorrect answer, as it is typically nocturnal or crepuscular (active at twilight).
* The response does not meet the instruction to provide three different animals

- レスポンスは非常に冗長であることに注意してください。どのモデルが優れているかを定量化するために、スコアのみを返したいと思います：

In [7]:
from tqdm import tqdm


def generate_model_scores(json_data, json_key):
    scores = []
    for entry in tqdm(json_data, desc="Scoring entries"):
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"score the model response `{entry[json_key]}`"
            f" on a scale from 0 to 100, where 100 is the best score. "
            f"Respond with the integer number only."
        )
        score = query_model(prompt)
        try:
            scores.append(int(score))
        except ValueError:
            continue

    return scores

- それでは、この評価をデータセット全体に適用し、各モデルの平均スコアを計算しましょう（M3 MacBook Airラップトップで1モデルあたり約1分かかります）
- 本記事執筆時点では、ollamaはオペレーティングシステム間で完全に決定論的ではないため、表示される数値は以下に示すものと若干異なる場合があることに注意してください

In [8]:
from pathlib import Path

for model in ("model 1 response", "model 2 response"):

    scores = generate_model_scores(json_data, model)
    print(f"\n{model}")
    print(f"Number of scores: {len(scores)} of {len(json_data)}")
    print(f"Average score: {sum(scores)/len(scores):.2f}\n")

    # Optionally save the scores
    save_path = Path("scores") / f"llama3-8b-{model.replace(' ', '-')}.json"
    with open(save_path, "w") as file:
        json.dump(scores, file)

Scoring entries: 100%|████████████████████████| 100/100 [01:02<00:00,  1.59it/s]



model 1 response
Number of scores: 100 of 100
Average score: 78.48



Scoring entries: 100%|████████████████████████| 100/100 [01:10<00:00,  1.42it/s]


model 2 response
Number of scores: 99 of 100
Average score: 64.98



- 上記の評価に基づくと、1番目のモデルは2番目のモデルよりも優れていると言えます